### Question 1
Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use _"lena.jfif"_ as the input image.
    - Use external libraries `imutils`.  
    - Modify the transformation matrix.

In [1]:
!pip install imutils

In [2]:
# Using imutils library
import cv2 as cv
import imutils

# Load the image
image = cv.imread('images/lena.jfif')

# Rotate the image by 45 degrees without cropping
rotated = imutils.rotate_bound(image, 45)

# Save the result
result_path = 'rotated_image.jpg'
save_success = cv.imwrite(result_path, rotated)

# Open the saved image using OpenCV
saved_image = cv.imread(result_path)

# Display the image in an OpenCV window
cv.imshow('Rotated Image', saved_image)
cv.waitKey(0)
cv.destroyAllWindows()

In [3]:
# Modify the Transformation Matrix
import cv2 as cv
import imutils
import numpy as np

# Load the image
image = cv.imread('images/lena.jfif')

(h, w) = image.shape[:2]
(cX, cY) = (w // 2, h // 2)
M = cv.getRotationMatrix2D((cX, cY), 45, 1.0)
cos = np.abs(M[0, 0])
sin = np.abs(M[0, 1])
nW = int((h * sin) + (w * cos))
nH = int((h * cos) + (w * sin))
M[0, 2] += (nW / 2) - cX
M[1, 2] += (nH / 2) - cY
rotated_matrix = cv.warpAffine(image, M, (nW, nH))
result_path_matrix = 'rotated_matrix_image.jpg'
save_success_matrix = cv.imwrite(result_path_matrix, rotated_matrix)
if not save_success_matrix:
    raise IOError('The image could not be saved using transformation matrix. Check the file path and permissions.')

# Display the image 
cv.imshow('Rotated Image (Transformation Matrix)', rotated_matrix)
cv.waitKey(0)
cv.destroyAllWindows()

### Question 2
Use the images with titles: _"flower.jfif"_ and _"native-bee.png"_. I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are `cv.bitwise_and()`, `cv.bitwise_or()` and `cv.bitwise_not()`. You need to use `cv.threshold` function to segment the flower. Please refer to [online documentation]

In [4]:
import cv2 as cv
import numpy as np

# Load images
flower = cv.imread('images/flower.jfif')
bee = cv.imread('images/native-bee.png')

# Resize flower to fit on bee image
flower = cv.resize(flower, (200, 200))

# Create a mask of the flower and its inverse mask
flower_gray = cv.cvtColor(flower, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(flower_gray, 1, 255, cv.THRESH_BINARY)
mask_inv = cv.bitwise_not(mask)

# Define the region of interest (ROI) on the bee image
rows, cols, _ = flower.shape
roi = bee[0:rows, 0:cols]

# Black-out the area of the flower in the ROI
bee_bg = cv.bitwise_and(roi, roi, mask=mask_inv)

# Take only the flower region from the flower image
flower_fg = cv.bitwise_and(flower, flower, mask=mask)

# Put the flower in the ROI and modify the bee image
dst = cv.add(bee_bg, flower_fg)
bee[0:rows, 0:cols] = dst

# Save the result
result_path = 'bee_with_flower.jpg'
cv.imwrite(result_path, bee)

# Display the image
cv.imshow('Bee with Flower', bee)
cv.waitKey(0)
cv.destroyAllWindows()

### Question 3
Write a function that randomly crop the central region of an image. The method signature should be as shown in the following:
```
random_center_crop(image, min_crop_ratio, max_crop_ratio)
```

In [5]:
import random

def random_center_crop(image, min_crop_ratio, max_crop_ratio):
    h, w = image.shape[:2]
    crop_ratio = random.uniform(min_crop_ratio, max_crop_ratio)
    crop_h = int(h * crop_ratio)
    crop_w = int(w * crop_ratio)
    
    start_x = (w - crop_w) // 2
    start_y = (h - crop_h) // 2
    
    cropped_image = image[start_y:start_y + crop_h, start_x:start_x + crop_w]
    return cropped_image

# Example usage
image = cv.imread('images/lena.jfif')

cropped_image = random_center_crop(image, 0.5, 0.8)
result_path = 'random_cropped_image.jpg'
cv.imwrite(result_path, cropped_image)

# Display the image
cv.imshow('Randomly Cropped Image', cropped_image)
cv.waitKey(0)
cv.destroyAllWindows()

### Question 4
Aside from Gaussian noise, name another common type of noise. Write the code to demonstrate how the noise can be included in an image.

ANS: Another common type of noise is Salt-and-Pepper Noise. This type of noise is also known as impulse noise and manifests as sparsely occurring white and black pixels in the image. It is typically caused by sharp, sudden disturbances in the image signal.

In [6]:
import cv2 as cv
import numpy as np

def add_salt_and_pepper_noise(image, salt_prob, pepper_prob):
    output = np.copy(image)
    h, w = image.shape[:2]
    
    # Add salt noise
    num_salt = np.ceil(salt_prob * h * w)
    coords = [np.random.randint(0, i, int(num_salt)) for i in image.shape[:2]]
    output[coords[0], coords[1]] = 255
    
    # Add pepper noise
    num_pepper = np.ceil(pepper_prob * h * w)
    coords = [np.random.randint(0, i, int(num_pepper)) for i in image.shape[:2]]
    output[coords[0], coords[1]] = 0
    
    return output

# Example usage
image = cv.imread('images/lena.jfif', cv.IMREAD_GRAYSCALE)

noisy_image = add_salt_and_pepper_noise(image, 0.02, 0.02)
result_path = 'salt_and_pepper_noise.jpg'
cv.imwrite(result_path, noisy_image)

# Display the image
cv.imshow('Salt and Pepper Noise', noisy_image)
cv.waitKey(0)
cv.destroyAllWindows()